In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：使用导出的数据集为在线预测训练自定义的表格分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_tabular_classification_online_exported_ds.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_tabular_classification_online_exported_ds.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来训练和部署一个自定义表格分类模型用于在线预测，使用一个导出的`Dataset`资源。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/iris)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/overview)。这个数据集不需要任何特征工程。您在本教程中将使用的数据集版本存储在一个公共Cloud Storage存储桶中。训练好的模型可以预测三种类型的鸢尾花品种：山鸢尾、维吉尼亚鸢尾或者变色鸢尾。

### 目标

在本教程中，您将学习如何使用从 Python 脚本中导出的 `Dataset` 资源在 Docker 容器中使用 Vertex 客户端库创建一个自定义模型，然后对部署的模型进行预测。您也可以选择使用 `gcloud` 命令行工具或在 Google Cloud 控制台上在线创建模型。

执行的步骤包括：

- 创建一个 Vertex `Dataset` 资源。
- 导出 `Dataset` 资源的清单。
- 创建一个用于训练模型的 Vertex 自定义作业。
- 导入导出的数据集清单。
- 训练模型。
- 检索并加载模型的工件。
- 查看模型评估。
- 将模型上传为 Vertex `Model` 资源。
- 部署 `Model` 资源到一个服务 `Endpoint` 资源。
- 进行预测。
- 取消部署 `Model` 资源。

### 成本

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新版本的 *google-cloud-storage* 库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

安装完Vertex客户端库和Google *云存储*后，您需要重新启动笔记本内核以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果有的话，请确保在 GPU 运行时下运行此笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您会获得$300 的免费信用额度用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对此笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 运行以`!`前缀的行作为 shell 命令，并将以`$`前缀的 Python 变量插值到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改`REGION`变量，该变量在笔记本其余部分中使用。以下是Vertex支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶来训练Vertex。并非所有区域均提供对所有Vertex服务的支持。有关各区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加现场教程会话，您可能正在使用共享的测试账户或项目。为了避免用户之间在创建的资源上发生名称冲突，您会为每个实例会话创建一个时间戳，并将其附加到本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的Google Cloud帐户

**如果您正在使用Google Cloud Notebook**，则您的环境已经得到验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**授予该服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，并选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，并选择**存储对象管理员**。

点击创建。一个包含您的密钥的JSON文件将下载到您的本地环境。

将您的服务帐户密钥的路径输入到下面的单元格中作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，都需要执行以下步骤。**

当您使用 Vertex 客户端库提交自定义训练作业时，您需要将包含训练代码的 Python 包上传到一个云存储桶中。Vertex 会从这个包中运行代码。在本教程中，Vertex 还会将您的作业生成的训练模型保存在同一个存储桶中。然后，您可以基于这个输出创建一个 `Endpoint` 资源，以便提供在线预测。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容验证对云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入顶点客户端库

将顶点客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

#### 顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的顶点 API 服务端点。
- `PARENT`：用于数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 标签常量

设置`Dataset`资源专属的常量进行标记

In [ ]:
# Tabular Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/tables_1.0.0.yaml"
# Tabular Labeling type
LABEL_SCHEMA = (
    "gs://google-cloud-aiplatform/schema/dataset/ioformat/table_io_format_1.0.0.yaml"
)

#### 硬件加速器

设置硬件加速器（例如，GPU）用于训练和预测。

设置变量`TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU` 来使用支持 GPU 的容器镜像，以及分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个 GPU 容器镜像，为每个 VM 分配 4 个 Nvidia Telsa K80 GPU，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`来使用一个在 CPU 上运行的容器镜像。

*注意*：TF 2.3 之前的版本对 GPU 支持将无法加载本教程中的自定义模型。这是一个已知问题，在 TF 2.3 中修复了，这是由于在服务函数中生成的静态图操作。如果你在自己的自定义模型上遇到这个问题，请使用支持 GPU 的 TF 2.3 容器镜像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）镜像

接下来，我们将设置用于训练和预测的Docker容器镜像

-TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
-TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
-TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
-TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
-TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
-XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
-Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
-Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请参见[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

-TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
-TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
-TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
-TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
-XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
-Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请参见[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：\ [2, 4, 8, 16, 32, 64, 96 \] 中的数量

*注意：以下机器类型不支持训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在你已经准备好开始为鸢尾花创建自己的定制模型并进行训练。

## 设置客户端

Vertex客户端库作为客户端/服务器模型运行。在您的一侧（Python脚本），您将创建一个客户端，用于向Vertex服务器发送请求并接收响应。

在本教程中，您将针对工作流程的不同步骤使用不同的客户端。因此，请提前设置它们。

- 数据集服务用于`数据集`资源。
- 模型服务用于`模型`资源。
- 端点服务用于部署。
- 作业服务用于批处理作业和自定义训练。
- 预测服务用于提供服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

数据集

现在您的客户已经准备好了，训练模型的第一步是创建一个托管的数据集实例，然后将标记数据上传到其中。

### 创建`Dataset`资源实例

使用辅助函数`create_dataset`来创建`Dataset`资源的实例。该函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个Vertex `Dataset`资源（`aip.Dataset`），具有以下参数：
 - `display_name`：您选择赋予的可读名称。
 - `metadata_schema_uri`：数据集类型的模式。
3. 调用客户端数据集服务方法`create_dataset`，具有以下参数：
 - `parent`：您的`Database`，`Model`和`Endpoint`资源的Vertex定位根路径。
 - `dataset`：您创建的Vertex数据集对象实例。
4. 该方法返回一个`operation`对象。

一个`operation`对象是Vertex处理长时间运行操作的异步调用方式。虽然这一步通常很快，但当您在项目中首次使用它时，由于需要预配，会有更长的延迟。

您可以使用`operation`对象来获取操作的状态（例如，创建`Dataset`资源）或取消操作，通过调用操作方法：

| 方法      | 描述 |
| ----------- | ----------- |
| result()    | 等待操作完成并以JSON格式返回结果对象。      |
| running()   | 返回操作是否仍在运行的True/False。        |
| done()      | 返回操作是否已完成的True/False。 |
| canceled()  | 返回操作是否已取消的True/False。 |
|cancel()    | 取消操作（可能需要高达30秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("iris-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### 数据准备

用于表格的Vertex `Dataset`资源对您的表格数据有一些要求。

- 必须在CSV文件或BigQuery查询中。

CSV

对于表格分类，CSV文件有一些要求：

- 第一行必须是标题--注意这与视觉、视频和语言不同，那里的要求是无标题。
- 除了一列之外，其他列都是特征。
- 一列是标签，您将在随后创建训练流程时指定。

#### 云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/tables/iris_1000.csv"

快速查看您的数据

您将使用存储在公共云存储桶中的鸢尾花数据集的一个版本，使用一个CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的Vertex数据集资源中。使用这个辅助函数 `import_data` 来导入数据。该函数执行以下操作：

- 使用 `Dataset` 客户端。
- 调用客户端方法 `import_data`，并传入以下参数：
 - `name`：您为 `Dataset` 资源指定的可读名称（例如，iris）。
 - `import_configs`：导入配置。

- `import_configs`：一个包含字典的 Python 列表，其中包含键/值条目：
 - `gcs_sources`：包含一个或多个索引文件路径的 URI 列表。
 - `import_schema_uri`：标识标记类型的模式。

`import_data()` 方法返回一个长时间运行的 `operation` 对象。这将需要几分钟才能完成。如果您在进行实时教程，现在是一个好时机提出问题或休息一下。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

### 导出数据集索引

接下来，您将把数据集索引导出到一个 JSONL 文件中，然后您的自定义训练工作将使用这个文件来获取数据和相应的标签，以训练您的鸢尾花模型。使用这个辅助函数 `export_data` 来导出数据集索引。该函数执行以下操作：

- 使用数据集客户端。
- 调用客户端方法 `export_data`，并提供以下参数：
 - `name`：您为数据集指定的可读名称（例如 iris）。
 - `export_config`：导出配置。
- `export_config`：包含一个字典的 Python 列表，其中包含以下键/值对：
 - `gcs_destination`：将 JSONL 数据集索引文件写入的 Cloud Storage 存储桶。

`export_data()` 方法返回一个长时间运行的 `operation` 对象。这将需要几分钟时间来完成。辅助函数将在导出完成时返回长时间运行的操作以及操作的结果。

In [ ]:
EXPORT_FILE = BUCKET_NAME + "/export"


def export_data(dataset_id, gcs_dest):
    config = {"gcs_destination": {"output_uri_prefix": gcs_dest}}
    start_time = time.time()
    try:
        operation = clients["dataset"].export_data(
            name=dataset_id, export_config=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation, result
    except Exception as e:
        print("exception:", e)
        return None, None


_, result = export_data(dataset_id, EXPORT_FILE)

## 训练模型

您可以使用容器镜像的两种方式来训练一个自定义模型：

- **使用谷歌云预构建的容器**。如果使用预构建的容器，您还需要指定要安装到容器镜像中的Python包。这个Python包包含了您用于训练自定义模型的代码。

- **使用您自己的自定义容器镜像**。如果使用您自己的容器，容器需要包含您用于训练自定义模型的代码。

## 准备您的自定义作业规格

现在您的客户准备好了，您的第一步是为您的自定义培训作业创建一个作业规格。 作业规格将包括以下内容：

- `worker_pool_spec`：用于培训的机器类型规格以及数量（单个或分布式）
- `python_package_spec`：预先构建的容器中要安装的Python软件包规范。

### 准备您的机器规格

现在为您的自定义训练工作定义机器规格。这告诉 Vertex 需要为训练提供哪种类型的机器实例。
- `machine_type`: 要提供的 GCP 实例的类型 -- 例如，n1-standard-8。
- `accelerator_type`: （如果有的话）硬件加速器的类型。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则表示您正在使用 GPU；否则您将使用 CPU。
- `accelerator_count`: 加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这将告诉 Vertex 在每台训练机器实例中提供哪种类型和大小的磁盘。

- `boot_disk_type`：SSD 或 Standard 两者之一。SSD 更快，Standard 较便宜。默认为 SSD。
- `boot_disk_size_gb`：磁盘的大小，以 GB 为单位。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

PARAM_FILE = BUCKET_NAME + "/params.txt"

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_iris.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 拼装工作规范

现在为自定义工作规范组装完整描述：

- `display_name`：您为此自定义工作分配的可读名称。
- `job_spec`：自定义工作的规范。
    - `worker_pool_specs`：机器VM实例的规格。
    - `base_output_directory`：这告诉服务在哪里保存模型工件的Cloud Storage位置（当变量`DIRECT = False`时）。然后服务将将位置传递给训练脚本作为环境变量`AIP_MODEL_DIR`，路径的格式将是：

               <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为自定义培训工作组装 Python 包。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到 Docker 镜像的操作环境的指令。

文件 `trainer/task.py` 是执行自定义训练工作的 Python 脚本。*注意*，当在工作人员池规范中引用它时，我们将目录斜杠替换为点（`trainer.task`）并省略文件后缀（`.py`）。

#### 包组装

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Iris tabular classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_iris.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用以下辅助函数`create_custom_job`，该函数接受以下参数：

-`custom_job`：自定义作业的规范。

辅助函数调用作业客户端服务的`create_custom_job`方法，并传入以下参数：

-`parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
-`custom_job`：自定义作业的规范。

您将显示在`response`对象中返回的一些字段，其中最感兴趣的两个字段是：

`response.name`：分配给此自定义训练作业的Vertex完全限定标识符。您需要保存此标识符以在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义工作的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

获取有关自定义作业的信息

接下来，使用这个辅助函数`get_custom_job`，它使用以下参数：

- `name`：自定义作业的Vertex完全合格的标识符。

辅助函数调用作业客户端服务的`get_custom_job`方法，使用以下参数：

- `name`：自定义作业的Vertex完全合格的标识符。

如果您记得，当您调用`create_custom_job`方法时，您在`response.name`字段中获取了自定义作业的Vertex完全合格标识符，并将标识符保存在变量`job_id`中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

# 部署

训练上述模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过从`end_time`减去`start_time`来计算训练模型所需的实际时间。对于您的模型，我们需要知道保存模型的位置，这个Python脚本保存在您的本地云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

## 加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在云存储桶中。现在从云存储桶加载它，然后您可以执行一些操作，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras `model.load_model()` 方法，将保存模型的云存储路径传递给它，由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型的好坏。

### 加载评估数据

你将从 `tf.keras.datasets` 加载鸢尾花测试数据（留出数据），使用 `load_data()` 方法。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：图像数据和对应的标签。

你不需要训练数据，因此我们将其加载为 `(_, _)`。

在运行数据进行评估之前，你需要对其进行预处理：

x_test:
1. 将像素数据进行归一化（重新缩放），通过将每个像素除以 255 来实现。这将用 0 到 1 之间的 32 位浮点数来替代每个单字节整数像素。

y_test:<br/>
2. 标签目前是标量（稀疏）的。如果你回顾 `trainer/task.py` 脚本中的 `compile()` 步骤，你会发现它是为稀疏标签编译的。因此我们无需再进行其他操作。

## 评估模型

现在让我们找出模型的好坏。

### 载入评估数据

您将从鸢尾花数据集的末尾加载一些样本数据，即最后10个项目。然后我们将对数据项进行预处理，形成：

- `x_test`：内存中的预处理图像数据。
- `y_test`：相应的标签。

In [ ]:
x_test = []
y_test = []

data_items = export_data_items[-10:]
for data_item in data_items:
    data_item = json.loads(data_item)
    print("FILE", data_item["imageGcsUri"])
    with tf.io.gfile.GFile(data_item["imageGcsUri"], "rb") as f:
        data = f.read()
    image = tf.io.decode_image(data)
    image = tf.image.resize(image, (128, 128))
    image = (image.numpy() / 255.0).astype(np.float32)
    cls = data_item["classificationAnnotation"]["displayName"]
    label = cls2label[cls]
    x_test.append(image)
    y_test.append(label)

x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

### 执行模型评估

现在评估定制作业中模型的表现如何。

In [ ]:
model.evaluate(x_test, y_test)

## 上传模型以供提供服务

接下来，您将从自定义作业中上传TF.Keras模型到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个提供函数，将数据转换为模型期望的格式。如果您发送编码数据到Vertex，则您的提供函数将确保数据在传递到模型之前在模型服务器上解码。

### 提供函数如何工作

当您向在线预测服务器发送请求时，请求将被一个HTTP服务器接收。HTTP服务器从HTTP请求内容主体中提取预测请求。提取的预测请求将被转发到提供函数。对于Google预构建的预测容器，请求内容将作为`tf.string`传递给提供函数。

提供函数由两部分组成：

- `预处理函数`：
  - 将输入（`tf.string`）转换为基础模型（动态图）的输入形状和数据类型。
  - 执行与训练基础模型期间相同的数据预处理 -- 例如，标准化、缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，创建JSON对象等。

预处理和后处理函数都会被转换为与模型融合的静态图。从基础模型的输出传递到后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

当为TF.Keras模型构建提供函数时，需要考虑一个重要因素，那就是它们作为静态图运行。这意味着，您不能使用需要动态图的TF图操作。如果您这样做，那么在编译提供函数时将会出现错误，并指出您正在使用不受支持的EagerTensor。

获取serving函数签名

您可以通过重新加载模型到内存，查询模型的输入和输出层的签名来获取模型的输入和输出层的签名。

当进行预测请求时，您需要将请求路由到serving函数而不是模型，因此您需要知道serving函数的输入层名称--这将在您进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用这个辅助函数`upload_model`来上传您的模型，保存为SavedModel格式，到`Model`服务中。`Model`服务会为您的模型实例化一个Vertex`Model`资源实例。一旦您完成了这一步，您可以像任何其他Vertex`Model`资源实例一样使用`Model`资源实例，比如部署到用于提供预测的`Endpoint`资源。

该辅助函数接受以下参数：

- `display_name`: 用于`Endpoint`服务的易读名称。
- `image_uri`: 用于模型部署的容器镜像。
- `model_uri`: 我们SavedModel artifact的Cloud Storage路径。对于本教程，这是`trainer/task.py`保存模型artifact的Cloud Storage位置，我们在变量`MODEL_DIR`中指定。

该辅助函数调用`Model`客户端服务的`upload_model`方法，该方法接受以下参数：

- `parent`: `Dataset`、`Model`和`Endpoint`资源的Vertex位置根路径。
- `model`: Vertex`Model`资源实例的规范。

现在让我们更深入地了解Vertex模型规范`model`。这是一个字典对象，包括以下字段：

- `display_name`: `Model`资源的易读名称。
- `metadata_schema_uri`: 因为您的模型是在没有Vertex`Dataset`资源的情况下构建的，所以将其留空（`''`）。
- `artifact_uri`: 保存模型以SavedModel格式存储的Cloud Storage路径。
- `container_spec`: 这是将安装在`Endpoint`资源上的Docker容器的规范，从中`Model`资源将提供预测。使用您之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源会返回一个长时间运行的操作，因为这可能需要一些时间。您调用response.result()，这是一个同步调用，当Vertex Model资源准备就绪时将返回。

该辅助函数返回相应Vertex Model实例upload_model_response.model的Vertex完全限定标识符。您将把该标识符保存在变量model_to_deploy_id中，以备后续步骤使用。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model("iris-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy)

### 获取 `Model` 资源信息

现在让我们获取只有您的模型的模型信息。使用这个辅助函数 `get_model`，带有以下参数：

- `name`：`Model` 资源的 Vertex 唯一标识符。

该辅助函数调用 Vertex `Model` 客户端服务的方法 `get_model`，带有以下参数：

- `name`：`Model` 资源的 Vertex 唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署`Model`资源

现在部署训练好的Vertex自定义`Model`资源。 这需要两个步骤：

1. 创建一个用于部署`Model`资源的`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

创建一个`Endpoint`资源

使用这个辅助函数`create_endpoint`来创建一个端点，用于部署模型以提供预测，传入以下参数：

- `display_name`：`Endpoint`资源的可读名称。

该辅助函数使用端点客户端服务的`create_endpoint`方法，该方法接受以下参数：

- `display_name`：`Endpoint`资源的可读名称。

创建`Endpoint`资源会返回一个长时间运行的操作，因为可能需要一些时间来为提供服务的`Endpoint`资源进行配置。您可以调用`response.result()`，这是同步调用，当Endpoint资源准备就绪时会返回。辅助函数返回`Endpoint`资源的Vertex完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "iris_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“端点”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例缩放

在处理您的在线预测请求时，您有几种选择可以进行计算实例的缩放：

- 单一实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动缩放：在线预测请求在您手动指定的固定数量的计算实例上分配。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同数量的节点。当模型首次部署到实例上时，会预先提供固定数量的计算实例，并且在线预测请求将均匀分布在这些实例之间。

- 自动缩放：在线预测请求在可扩展数量的计算实例上分配。
  - 在模型首次部署时，将计算实例的最小（`MIN_NODES`）数量设定为预留和取消预留的实例数量，并且根据负载情况，将计算实例的最大（`MAX_NODES`）数量设定为要提供的实例数量。

计算实例的最小数量对应于字段`min_replica_count`，计算实例的最大数量对应于字段`max_replica_count`，在您后续的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用这个辅助函数 `deploy_model` 来将从训练管道中上传（部署）的 `Model` 资源部署到您为提供预测创建的 `Endpoint` 资源，使用以下参数：

- `model`: 要上传（部署）的模型的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`: 部署模型的可读名称。
- `endpoint`: 要部署模型的 Vertex 完全限定终端标识符。

辅助函数调用 `Endpoint` 客户端服务的方法 `deploy_model`，该方法接受以下参数：

- `endpoint`: 要将 `Model` 资源部署到的 Vertex 完全限定 `Endpoint` 资源标识符。
- `deployed_model`: 部署模型的要求规范。
- `traffic_split`: 流量分配到终端的百分比，作为一个包含一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 表示上传的这个模型，100 表示流量的 100%。
   - 如果终端上有现有模型，流量将被拆分到这些模型，使用 `model_id` 指定为 **{ "0": 百分比, model_id: 百分比, ... }**，其中 `model_id` 是要部署到终端的现有模型的模型标识符。百分比必须相加得到 100。

现在让我们深入了解 `deployed_model` 参数。该参数被指定为一个具有最少必填字段的 Python 字典：

- `model`: 要上传的 Vertex 完全限定模型标识符。
- `display_name`: 部署模型的可读名称。
- `disable_container_logging`: 关闭容器事件的记录，如执行失败（默认情况下容器日志记录已启用）。容器日志记录通常在调试部署时启用，然后在生产中部署时停用。
- `dedicated_resources`: 指定为用于提供预测请求的计算实例（副本）的数量。
  - `machine_spec`: 要提供的计算实例。使用之前设置的变量 `DEPLOY_GPU != None` 可以使用 GPU；否则只分配 CPU。
  - `min_replica_count`: 最初要提供的计算实例数量，之前设置为变量 `MIN_NODES`。
  - `max_replica_count`: 要扩展到的最大计算实例数量，之前设置为变量 `MAX_NODES`。

#### 流量分配

现在让我们深入了解 `traffic_split` 参数。该参数被指定为一个 Python 字典。起初可能会有些困惑。让我解释一下，您可以将您的模型的多个实例部署到一个终端，并设置每个实例分配的百分比。

为什么要这样做？也许您已经在生产中部署了先前版本 -- 让我们称其为 v1。您在 v2 上得到了更好的模型评估，但在将其部署到生产中之前，您不确定它是否真的更好。所以在流量分配的情况下，您可能希望将 v2 部署到与 v1 相同的终端上，但只占流量的10%。这样，您可以在不影响大多数用户的情况下监视其性能 -- 直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 来等待操作完成，这将阻止直到模型部署完毕。如果这是第一次将模型部署到终端，可能需要额外几分钟来完成资源的配置。

In [ ]:
DEPLOYED_NAME = "iris_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

发出一个在线预测请求

现在对您部署的模型进行在线预测。

In [ ]:
# Last data item in exported dataset index
data_items = export_data_items[-1:]
data_item = json.loads(data_items[0])
image_path = data_item["imageGcsUri"]
print("IMAGE PATH", image_path)

准备请求内容
您将发送图像作为压缩的JPG图像，而不是原始未压缩的字节：

- `cv2.imread`：将压缩的JPG图像读入内存作为原始字节。
- `tf.image.resize`：将图像调整大小为模型的输入形状--（128, 128, 3）。
- cv2.imwrite：将调整大小后的图像写回磁盘。
- `base64.b64encode`：读取调整大小后的压缩图像并编码为一个base 64编码的字符串。

In [ ]:
! gsutil cp $image_path tmp.jpg

import base64

import cv2

test_image = cv2.imread("tmp.jpg", cv2.IMREAD_COLOR)
print("before:", test_image.shape)
test_image = cv2.resize(test_image, (128, 128))
print("after:", test_image.shape)
cv2.imwrite("tmp.jpg", test_image.astype(np.uint8))

# bytes = tf.io.read_file('tmp.jpg')
with open("tmp.jpg", "rb") as f:
    bytes = f.read()
b64str = base64.b64encode(bytes).decode("utf-8")

### 发送预测请求

好的，现在您有一个测试数据项。使用这个辅助函数 `predict_data`，它接受以下参数：

- `data`: 测试数据项，作为一个浮点值的 numpy 1D 数组。
- `endpoint`: Vertex 完全限定标识符，指向部署了 `Model` 资源的 `Endpoint` 资源。
- `parameters_dict`: 用于服务的额外参数。

该函数使用预测客户端服务，并使用以下参数调用 `predict` 方法：

- `endpoint`: Vertex 完全限定标识符，指向部署了 `Model` 资源的 `Endpoint` 资源。
- `instances`: 需要预测的实例（数据项）的列表。
- `parameters`: 用于服务的额外参数。

为了将测试数据传递给预测服务，您需要将它打包以便传输到服务二进制文件，步骤如下：

    1. 将数据项从 1D numpy 数组转换为 1D Python 列表。
    2. 将预测请求转换为序列化的 Google protobuf (`json_format.ParseDict()`)

每个预测请求中的实例都是一个字典项，形式如下：

                        {input_name: content}

- `input_name`: 底层模型的输入层名称。
- `content`: 数据项作为 1D Python 列表。

由于 `predict()` 服务可以接受多个数据项（实例），您将把单个数据项作为一个数据项的列表发送。最后，您需要将实例列表打包成 Google 的 protobuf 格式 -- 这是传递给 `predict()` 服务的内容。

`response` 对象返回一个列表，其中列表中的每个元素对应请求中的相应图像。对于每个预测，您将在输出中看到：

- `predictions` -- 以 1K USD 为单位的房屋预测中位数值。

In [ ]:
def predict_data(data, endpoint, parameters_dict):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: data.tolist()}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_data(test_item, endpoint_id, None)

## 取消部署 `Model` 资源

现在从提供服务的 `Endpoint` 资源中取消部署您的 `Model` 资源。使用这个辅助函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：当 `Model` 资源部署到时，端点服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 Vertex 完全合格的 `Endpoint` 资源标识符。

此函数调用端点客户端服务的方法 `undeploy_model`，使用以下参数：

- `deployed_model_id`：端点服务在 `Model` 资源部署时返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 Vertex 完全合格的 `Endpoint` 资源标识符。
- `traffic_split`：如何在 `Endpoint` 资源上分配流量给剩余的部署模型。

因为这是 `Endpoint` 资源上唯一部署的模型，您可以通过将其设置为空对象 `{}` 而简单地将 `traffic_split` 留空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理本项目中使用的所有GCP资源，可以删除用于教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的单个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME